# Prototype calculation of cabelling and mixing of density from LLC4320 data

In [12]:
import xarray as xr
import numpy as np
import dask.array as dsa
import matplotlib.pyplot as plt
from fastjmd95 import jmd95numba
import pop_tools
import gcsfs
import xmitgcm
from xmitgcm.llcreader.llcmodel import faces_dataset_to_latlon

In [23]:
from intake import open_catalog

url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml"
cat = open_catalog(url)
sst  = cat["LLC4320_SST"].to_dask()
sss  = cat["LLC4320_SSS"].to_dask()
grid = cat["LLC4320_grid"].to_dask()

In [29]:
xr.DataArray({'sst':sst}, coords=grid)

ValueError: different number of dimensions on data and dims: 0 vs 7

In [26]:
ds = xr.merge([sst, grid, sss.drop_dims(['i_g', 'j_g', 'k', 'k_l', 'k_p1', 'k_u']).drop_vars('iter')])

MergeError: unable to determine if these variables should be coordinates or not in the merged result: {'XC', 'YC'}

In [ ]:
ds_ll = faces